In [ ]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.prepare import model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

In [ ]:
params_path = './Models'
product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])

In [ ]:
n = 0
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

In [ ]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

if len(df) == 0:
    raise ValueError("No data!")

df, input_features = preprocessData(df, 'date', target)

In [ ]:
data = df.copy()

data['time_idx'] = range(len(data))
data['group'] = product

training_cutoff = floor(len(data) * 0.8)

max_prediction_length = 14
max_encoder_length = 60 # 7, 14, 30, 60, 120
batch_size = 64

group = ['group']
# time_varying_known_categoricals = ['month', 'week']
time_varying_known_categoricals = []
time_varying_unknown_categoricals = []
time_varying_known_reals = ['time_idx']
time_varying_unknown_reals = input_features + [target]

In [ ]:
import warnings, torch, shutil
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
pl.seed_everything(123)

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, DeepAR, RecurrentNetwork, GroupNormalizer
from pytorch_forecasting.metrics import MAPE, NormalDistributionLoss, QuantileLoss, SMAPE

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from AnalyzeTools.prepare import retriveBestModelPath, save_best_params
from AnalyzeTools.paramsTuner import optimize_hyperparameters_for_RNN, optimize_hyperparameters_for_DeepAR

def DEEPAR(
    data, 
    training_cutoff, 
    target, 
    group, 
    max_encoder_length, 
    max_prediction_length, 
    time_varying_known_categoricals,
    time_varying_unknown_categoricals, 
    time_varying_known_reals, 
    batch_size,
    saving_dir,
    training_params={},
):
    best_model_path = retriveBestModelPath(saving_dir)
    data[time_varying_known_categoricals] = data[time_varying_known_categoricals].astype(str).astype("category")
    max_epochs = training_params.get('max_epochs') if training_params.get('max_epochs') else 100
    n_trials = training_params.get('n_trials') if training_params.get('n_trials') else 30

    data[time_varying_known_categoricals] = data[time_varying_known_categoricals].astype(str).astype("category")
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target=target,
        group_ids=group,
        max_encoder_length=max_encoder_length,
        max_prediction_length=max_prediction_length,
        time_varying_known_categoricals=time_varying_known_categoricals,
        time_varying_unknown_categoricals=time_varying_unknown_categoricals,
        time_varying_known_reals=time_varying_known_reals,
        time_varying_unknown_reals=[target],
        target_normalizer=GroupNormalizer(groups=group),
        categorical_encoders={
            "month": NaNLabelEncoder().fit(data.month),
            "week": NaNLabelEncoder().fit(data.week)
        },
    )

    validation = TimeSeriesDataSet.from_dataset(
        training, 
        data, 
        min_prediction_idx=training.index.time.max() + 1 + max_encoder_length - 1,
        stop_randomization=True
    )

    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    if not best_model_path:

        # create study
        study = optimize_hyperparameters_for_DeepAR(
            train_dataloader,
            val_dataloader,
            model_path=saving_dir,
            n_trials=n_trials,
            max_epochs=max_epochs,
            gradient_clip_val_range=(0.01, 1.0),
            hidden_size_range=(8, 128),
            learning_rate_range=(0.001, 0.1),
            dropout_range=(0.1, 0.3),
            trainer_kwargs=dict(limit_train_batches=30),
            reduce_on_plateau_patience=4,
            use_learning_rate_finder=False,
            log_dir=saving_dir
        )

        best_params = study.best_params
        shutil.rmtree(saving_dir)

        early_stop_callback = EarlyStopping(monitor="val_loss", verbose=False, mode="min")
        lr_logger = LearningRateMonitor()  # log the learning rate

        trainer = pl.Trainer(
            max_epochs=max_epochs,
            gpus=0,
            enable_model_summary=False,
            callbacks=[lr_logger, early_stop_callback],
            log_every_n_steps=10,
            check_val_every_n_epoch=3, 
            default_root_dir=saving_dir,
        )

        deep_ar = DeepAR.from_dataset(
            training,
            hidden_size=best_params.get("hidden_size"),
            rnn_layers=best_params.get('rnn_layers'),
            learning_rate=best_params.get('learning_rate'),
            dropout=best_params.get("dropout"),
            loss=NormalDistributionLoss(),
            log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
            log_val_interval=3,
        )
        # print(f"Number of parameters in network: {deep_ar.size()/1e3:.1f}k")

        trainer.fit(
            deep_ar,
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader,
        )

        save_best_params(best_params, f"{saving_dir}/Best_DeepAR.json")

    best_model_path = retriveBestModelPath(saving_dir)
    deep_ar = DeepAR.load_from_checkpoint(best_model_path)

    return deep_ar, val_dataloader

In [ ]:
dl_searchCV_params = {
    'base_dir': params_path,
    'product_and_product_type': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'predict_type': 'multiple',
    'period': 'Day',
    'step': max_prediction_length,
    'save': True
}

In [ ]:
# print("\nDeepAR")
# training_params = {'max_epochs': 10, 'n_trials': 2}
# saving_dir = pathForSavingModels('DeepAR', **dl_searchCV_params)
# deep_ar, val_dataloader = DEEPAR(
#     data,
#     training_cutoff,
#     target,
#     group,
#     max_encoder_length,
#     max_prediction_length,
#     time_varying_known_categoricals,
#     time_varying_unknown_categoricals,
#     time_varying_known_reals,
#     batch_size,
#     saving_dir,
#     training_params
# )

# actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
# deepar_predictions = deep_ar.predict(val_dataloader)



In [ ]:
print("\nTFT")
training_params = {'max_epochs': 10, 'n_trials': 1}
saving_dir = pathForSavingModels('TFT', **dl_searchCV_params)
tft, val_dataloader = TFT(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_unknown_categoricals,
    time_varying_known_categoricals,
    time_varying_known_reals,
    time_varying_unknown_reals,
    batch_size,
    saving_dir,
    training_params,
    predict_type='multiple'
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
tft_predictions = tft.predict(val_dataloader)

# model_eval(actuals, tft_predictions, predictions_x_axis, stdout=True, vis=True)

In [ ]:
model = tft

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = model.predict(val_dataloader)
(actuals - predictions).abs().mean()

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions, x = model.predict(val_dataloader, mode='raw', return_x=True, n_samples=10)

In [ ]:
quantiles_predictions, x = model.predict(val_dataloader, mode='quantiles', return_x=True, n_samples=10)

In [ ]:
quantiles_predictions[0]

In [ ]:
# series = validation.x_to_index(x)["series"]
for idx in range(20):  # plot 10 examples
    model.plot_prediction(x, predictions, idx=idx, add_loss_to_title=True)
    # plt.suptitle(f"Series: {series.iloc[idx]}")

In [ ]:
cnt = 0
for idx in range(1, 2000, max_prediction_length):  # plot 10 examples
    if cnt >= 50:
        break
    try:
        model.plot_prediction(x, predictions, idx=idx, add_loss_to_title=True)
        cnt += 1
    except IndexError:
        break

In [ ]:
interpretation = model.interpret_output(predictions, reduction="sum")
model.plot_interpretation(interpretation)